In [1]:
import os
import pandas as pd
import json
from pandarallel import pandarallel
import requests

In [2]:
os.chdir('/mnt/d/workspace/8-2/Financial-Statements-Text-Analysis/')

In [3]:
# params
with open('config.json', 'r') as f:
    c = json.load(f)
destination_dir = os.path.join(c['DATA_DIR'], '10k_raw')

In [4]:
# read index
metadata = pd.read_csv('data/metadata_2017.csv')
# cut to 100 for test purposes
metadata = metadata.head(100)

# Download filing basic functionality

In [22]:
def download_filing(url, destination_file, skip_existing=True, skip_ixbrl=True):
    """
    Given a SEC EDGAR 10-K URL, will download the HTML file to the local destination file.
    If skip, then will not re-download file if the local destination file already exists.
    """
    try:
        destination_dir = os.path.dirname(destination_file)

        if not os.path.isdir(destination_dir):
            os.makedirs(destination_dir)

        if skip_existing and os.path.exists(destination_file):
            print('⏭️ already exists, skipping download: {url}'.format(
            url=url))
            return

        # do not download iXBRL output
        if skip_ixbrl:
            url = url.replace('ix?doc=/', '')
        file_content = requests.get(url, headers={"User-Agent":"New York University eh1885@nyu.edu"}).text

        with open(destination_file, "w") as f:
            f.write(file_content)
        
        return file_content

    except:
        print('❌ downloaded failed: {url}'.format(
            url=url))

# Sequential download

In [6]:
%%timeit
for i in range(len(metadata)):
    row = metadata.iloc[i]
    ticker = row['TICKER']
    url = 'https://www.sec.gov/Archives/' + row['EDGAR_LINK']
    file_name = url.split("/")[-1] 
    destination_file = os.path.join(destination_dir, ticker, file_name)

    download_filing(url, destination_file, skip_existing=False)

8.32 s ± 967 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Parallelized download

In [7]:
%%timeit
number_of_workers = 4
pandarallel.initialize(progress_bar=True, nb_workers=number_of_workers, verbose=0)

def pandarallel_wrapper(metadata):
    """
    Basic wrapper of the download_filing functionality to allow for pandarallel optimization
    """
    ticker = metadata['TICKER']
    url = 'https://www.sec.gov/Archives/' + metadata['EDGAR_LINK']
    file_name = url.split("/")[-1] 
    destination_file = os.path.join(destination_dir, ticker, file_name)

    download_filing(url, destination_file, skip_existing=False)

metadata.parallel_apply(pandarallel_wrapper, axis=1)

3.29 s ± 155 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
number_of_workers = 16
pandarallel.initialize(progress_bar=True, nb_workers=number_of_workers, verbose=0)

def pandarallel_wrapper(metadata):
    """
    Basic wrapper of the download_filing functionality to allow for pandarallel optimization
    """
    ticker = metadata['TICKER']
    url = 'https://www.sec.gov/Archives/' + metadata['EDGAR_LINK']
    file_name = url.split("/")[-1] 
    destination_file = os.path.join(destination_dir, ticker, file_name)

    download_filing(url, destination_file, skip_existing=False)

metadata.parallel_apply(pandarallel_wrapper, axis=1)

2.35 s ± 371 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# Batch sequential download
Increments of 10 seconds

In [28]:
 # read URL table
import time
import re

metadata = pd.read_csv(os.path.join(c['DATA_DIR'], 'metadata.csv'))
metadata_legacy = pd.read_csv(os.path.join(c['DATA_DIR'], 'metadata_2017.csv'))

# only download the data from russell 3000 today
metadata = metadata_legacy[metadata_legacy['TICKER'].isin(metadata['ticker'])]
pattern = r"Your request rate has exceeded the SEC"
pattern_2 = r"To allow for equitable access to all users"

import time
start = time.time()
for i in range(len(metadata)):
    if i % 9 == 0:
        time.sleep(1)
    row = metadata.iloc[i]
    ticker = row['TICKER']
    url = 'https://www.sec.gov/Archives/' + row['EDGAR_LINK']
    file_name = url.split("/")[-1] 
    
    destination_file = os.path.join(destination_dir, ticker, file_name)

    retry = True
    
    while retry:
        filing = download_filing(url, destination_file, skip_existing=False)
        match = re.findall(pattern, filing)
        if len(match) == 0:
            retry = False
        else:
            print('Rate limited for file {} and ticker {}'.format(url, ticker))
            time.sleep(600)
    print('Successfully downloaded for file {} and ticker {}'.format(url, ticker))

Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1000228/0001000228-13-000010.txt and ticker HSIC
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1000694/0001144204-13-014545.txt and ticker NVAX
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1000697/0001193125-13-076804.txt and ticker WAT
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1000753/0001140361-13-005779.txt and ticker NSP
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1001082/0001104659-13-011967.txt and ticker DISH
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1001233/0001193125-13-076887.txt and ticker SGMO
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1001316/0001144204-13-016691.txt and ticker TGTX
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1001385/0001193125-13-112587.txt and ticker NWPX
Successfully downloaded fo

Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1035002/0001035002-13-000008.txt and ticker VLO
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1035092/0001144204-13-015234.txt and ticker SHBI
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1035201/0001047469-13-001968.txt and ticker CWT
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1035267/0001193125-13-036675.txt and ticker ISRG
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1035443/0001104659-13-014040.txt and ticker ARE
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1035688/0001047469-13-001696.txt and ticker SEM
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1035983/0001047469-13-001962.txt and ticker FIX
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/103730/0000103730-13-000009.txt and ticker VSH
Successfully downloaded for fil

Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1057352/0001057352-13-000013.txt and ticker CSGP
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/105770/0000105770-13-000007.txt and ticker WST
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1057877/0001057877-13-000040.txt and ticker IDA
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1058057/0001193125-13-134707.txt and ticker MRVL
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1058090/0001193125-13-046377.txt and ticker CMG
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1058290/0001193125-13-077271.txt and ticker CTSH
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1059556/0001193125-13-074571.txt and ticker MCO
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1060391/0001060391-13-000011.txt and ticker RSG
Successfully downloaded for fi

Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1100682/0001445305-13-000358.txt and ticker CRL
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1101239/0001193125-13-076572.txt and ticker EQIX
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1101302/0001193125-13-071771.txt and ticker ENTG
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1102112/0001047469-13-001982.txt and ticker PACW
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1102993/0001144204-13-012330.txt and ticker LPSN
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1103982/0001193125-13-073227.txt and ticker MDLZ
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1104506/0001140361-13-012797.txt and ticker INSM
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1104657/0001104657-13-000047.txt and ticker MTRN
Successfully downloaded f

Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1162461/0001140361-13-012682.txt and ticker CUTR
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1163165/0001193125-13-065426.txt and ticker COP
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1163302/0001193125-13-061613.txt and ticker X
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1163739/0001104659-13-016655.txt and ticker NBR
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1164727/0001193125-13-070256.txt and ticker NEM
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1164863/0001193125-13-080164.txt and ticker NPO
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1166003/0001193125-13-103438.txt and ticker XPO
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1166388/0001166388-13-000056.txt and ticker VRNT
Successfully downloaded for file

...

Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/108385/0000108385-16-000127.txt and ticker WRLD
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1124524/0001144204-16-110227.txt and ticker CYRX
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1169561/0001169561-16-000029.txt and ticker CVLT
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1170010/0001170010-16-000117.txt and ticker KMX
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/12659/0001574842-16-000042.txt and ticker HRB
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1289308/0001289308-16-000060.txt and ticker ENS
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1295947/0001295947-16-000051.txt and ticker PBH
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1302215/0001302215-16-000050.txt and ticker HLI
Successfully downloaded for file

Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1024305/0001024305-16-000181.txt and ticker COTY
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1039065/0001047469-16-015059.txt and ticker OSIS
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1048911/0001193125-16-650267.txt and ticker FDX
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1049521/0001049521-16-000094.txt and ticker MRCY
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1058057/0001193125-16-653778.txt and ticker MRVL
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/106040/0000106040-16-000033.txt and ticker WDC
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1067294/0001140361-16-080733.txt and ticker CBRL
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1069899/0001571049-16-017853.txt and ticker PAHC
Successfully downloaded for

Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/817720/0001564590-16-024739.txt and ticker SYNA
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/842023/0001437749-16-038182.txt and ticker TECH
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/85535/0001047469-16-014916.txt and ticker RGLD
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/855654/0001558370-16-008099.txt and ticker IMGN
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/858877/0000858877-16-000117.txt and ticker CSCO
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/866729/0000866729-16-000030.txt and ticker SCHL
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/8670/0000008670-16-000053.txt and ticker ADP
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/880117/0001193125-16-689942.txt and ticker JBSS
Successfully downloaded for file htt

Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1670541/0001670541-16-000016.txt and ticker ADNT
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/1674910/0001674910-16-000004.txt and ticker VVV
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/21510/0000021510-16-000074.txt and ticker COHR
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/22444/0000022444-16-000059.txt and ticker CMC
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/23197/0000023197-16-000074.txt and ticker CMTL
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/2969/0001193125-16-773346.txt and ticker APD
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/315189/0001047469-16-017244.txt and ticker DE
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/320193/0001628280-16-020309.txt and ticker AAPL
Successfully downloaded for file https:/

Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/909832/0000909832-16-000032.txt and ticker COST
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/912603/0000912603-16-000262.txt and ticker SCHN
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/914712/0001193125-16-738725.txt and ticker MLNK
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/915840/0000915840-16-000135.txt and ticker BZH
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/923120/0001193125-16-746547.txt and ticker GBX
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/924717/0001564590-16-029755.txt and ticker SRDX
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/935419/0001493152-16-015847.txt and ticker RICK
Successfully downloaded for file https://www.sec.gov/Archives/edgar/data/936395/0000936395-16-000128.txt and ticker CIEN
Successfully downloaded for file h

In [27]:
len(metadata)

6380